# Old PhenoDigm evidence


There are a few things I need to check

1. A given mouse model can be mapped to different disease/target associations.
2. Check if a mouse and human phenotypes the same for every evidence based on a given model

In [7]:
import gzip
import json
import pandas as pd


phenodigm_file = '/Users/dsuveges/project_data/ot/evidence_input/20.11/phenodigm/phenodigm-29-10-2020.json.gz'

parsed_data = []
with gzip.open(phenodigm_file, 'r') as f:
    for row in f:
        data = json.loads(row)
        parsed_data.append({
            'target' : data['target']['id'].split('/')[-1],
            'disease' : data['disease']['id'].split('/')[-1],
            'disease_label' : data['disease']['name'],
            'model_id' : data['evidence']['biological_model']['model_id'],
            'mouse_phenotypes' : [x['id'] for x in data['evidence']['biological_model']['phenotypes']],
            'human_phenotypes': [x['id'] for x in data['evidence']['disease_model_association']['human_phenotypes']]
        })

print(len(parsed_data))
phenodigm_df = pd.DataFrame(parsed_data)

print(phenodigm_df.head())

687439
            target         disease  \
0  ENSG00000115977  Orphanet_79235   
1  ENSG00000115977    Orphanet_205   
2  ENSG00000115977     EFO_1000946   
3  ENSG00000115977     EFO_0003888   
4  ENSG00000115977  Orphanet_88616   

                                       disease_label               model_id  \
0                     Crigler-Najjar syndrome type 2  MGI:5766745#hom#early   
1                            Crigler-Najjar syndrome  MGI:5766745#hom#early   
2                        gastric mucosal hypertrophy  MGI:5766745#hom#early   
3           attention deficit hyperactivity disorder  MGI:5766745#hom#early   
4  Autosomal recessive non-syndromic intellectual...  MGI:5766745#hom#early   

                                    mouse_phenotypes          human_phenotypes  
0  [MP:0005567, MP:0003795, MP:0001399, MP:001188...              [HP:0008282]  
1  [MP:0005567, MP:0003795, MP:0001399, MP:001188...              [HP:0008282]  
2  [MP:0005567, MP:0003795, MP:0001399, MP:001

In [9]:
print(f'Number of evidence: {len(phenodigm_df)}')
print(f"Number of disease/target association: {len(phenodigm_df[['target','disease']].drop_duplicates())}")
print(f"Number of mouse models: {len(phenodigm_df.model_id.unique())}")
print(f"Number of unique targets: {len(phenodigm_df.target.unique())}")
print(f"Number of unique diseases: {len(phenodigm_df.disease.unique())}")

Number of evidence: 687439
Number of disease/target association: 376633
Number of mouse models: 21862
Number of unique targets: 8644
Number of unique diseases: 4693


In [34]:
xf = (
    phenodigm_df
    .groupby('model_id')
    .agg(
        dict(mouse_phenotypes = lambda x: x.apply(json.dumps).unique().shape[0],
        human_phenotypes = lambda x: x.apply(json.dumps).unique().shape[0])
    )
)

In [36]:
print(f"Number of ")
xf.head()

array([1])